# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_a
　　セッション1のAさんのスケーリングデータ　→　data1_a_label_50

In [2]:
%time
#sensor_dataの読み込み

# data1_a = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_a = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

#data1_a_label_50 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_50.csv')
#data1_a_label_60 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_60.csv')
#data1_a_label_70 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_70.csv')
#data1_a_label_80 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_80.csv')

data2_a_label_50 = pd.read_csv('emotion_label/session2/B/emotion_A_20201111_2_50.csv')
# data2_a_label_60 = pd.read_csv('emotion_label/session2/B/emotion_A_20201111_2_60.csv')
# data2_a_label_70 = pd.read_csv('emotion_label/session2/B/emotion_A_20201111_2_70.csv')
# data2_a_label_80 = pd.read_csv('emotion_label/session2/B/emotion_A_20201111_2_80.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [496]:
data2_a_label_70

,start_time,finish_time,emotion_level
0,3:27,3:31,0
1,3:41,3:45,-2
2,3:42,3:46,-2
3,3:43,3:47,-2
4,3:44,3:48,-2
...,...,...,...
459,18:06,18:10,-2
460,18:07,18:11,-2
461,18:08,18:12,-2
462,18:09,18:13,-2


In [497]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

#data1_a_label_50 = dropna(data1_a_label_50)
#data1_a_label_60 = dropna(data1_a_label_60)
#data1_a_label_70 = dropna(data1_a_label_70)
#data1_a_label_80 = dropna(data1_a_label_80)

# data2_a_label_50 = dropna(data2_a_label_50)
# data2_a_label_60 = dropna(data2_a_label_60)
data2_a_label_70 = dropna(data2_a_label_70)
# data2_a_label_80 = dropna(data2_a_label_80)

In [498]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=0)
    return a

#data1_a_label_50 = dropna(data1_a_label_50)
#data1_a_label_60 = dropna(data1_a_label_60)
#data1_a_label_70 = dropna(data1_a_label_70)
#data1_a_label_80 = dropna(data1_a_label_80)

# data2_a_label_50 = dropna(data2_a_label_50)
# data2_a_label_60 = dropna(data2_a_label_60)
data2_a_label_70 = dropna(data2_a_label_70)
# data2_a_label_80 = dropna(data2_a_label_80)

In [499]:
data2_a_label_70

,start_time,finish_time,emotion_level
0,3:27,3:31,0
1,3:41,3:45,-2
2,3:42,3:46,-2
3,3:43,3:47,-2
4,3:44,3:48,-2
...,...,...,...
459,18:06,18:10,-2
460,18:07,18:11,-2
461,18:08,18:12,-2
462,18:09,18:13,-2


In [500]:
#生データの確認
display(data2_a)
display(data2_a_label_70)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,14:57:59.245,9801,898,-258,340,1,913
1,ags,14:57:59.250,9911,592,-290,291,44,559
2,ags,14:57:59.255,9996,431,-268,218,-188,279
3,ags,14:57:59.260,10150,453,-280,114,-84,-75
4,ags,14:57:59.265,10292,263,-122,-43,-267,-415
...,...,...,...,...,...,...,...,...
469692,ags,15:37:07.780,10184,356,313,-318,860,-20
469693,ags,15:37:07.785,10216,282,230,-190,1006,-117
469694,ags,15:37:07.790,10087,336,339,-68,1116,-208
469695,ags,15:37:07.795,10284,273,331,23,1134,-287


,start_time,finish_time,emotion_level
0,3:27,3:31,0
1,3:41,3:45,-2
2,3:42,3:46,-2
3,3:43,3:47,-2
4,3:44,3:48,-2
...,...,...,...
459,18:06,18:10,-2
460,18:07,18:11,-2
461,18:08,18:12,-2
462,18:09,18:13,-2


In [501]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data2_a)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
# norm_order(data2_a)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [502]:
data2_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [503]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

#datetime_order(data1_a)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

datetime_order(data2_a)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [504]:
data2_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,2020-11-11 14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,2020-11-11 14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,2020-11-11 14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,2020-11-11 14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,2020-11-11 15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,2020-11-11 15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,2020-11-11 15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,2020-11-11 15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


In [505]:
#data1_a_label_50['start_time'] = pd.to_datetime(data1_a_label_50['start_time'], format='%M:%S')
#data1_a_label_50['finish_time'] = pd.to_datetime(data1_a_label_50['finish_time'], format='%M:%S')

In [506]:
#data1_a_label_60['start_time'] = pd.to_datetime(data1_a_label_60['start_time'], format='%M:%S')
#data1_a_label_60['finish_time'] = pd.to_datetime(data1_a_label_60['finish_time'], format='%M:%S')

In [507]:
#data1_a_label_70['start_time'] = pd.to_datetime(data1_a_label_70['start_time'], format='%M:%S')
#data1_a_label_70['finish_time'] = pd.to_datetime(data1_a_label_70['finish_time'], format='%M:%S')

In [508]:
#data1_a_label_80['start_time'] = pd.to_datetime(data1_a_label_80['start_time'], format='%M:%S')
#data1_a_label_80['finish_time'] = pd.to_datetime(data1_a_label_80['finish_time'], format='%M:%S')

In [509]:
# data2_a_label_50['start_time'] = pd.to_datetime(data2_a_label_50['start_time'], format='%M:%S')
# data2_a_label_50['finish_time'] = pd.to_datetime(data2_a_label_50['finish_time'], format='%M:%S')

In [510]:
# data2_a_label_60['start_time'] = pd.to_datetime(data2_a_label_60['start_time'], format='%M:%S')
# data2_a_label_60['finish_time'] = pd.to_datetime(data2_a_label_60['finish_time'], format='%M:%S')

In [511]:
data2_a_label_70['start_time'] = pd.to_datetime(data2_a_label_70['start_time'], format='%M:%S')
data2_a_label_70['finish_time'] = pd.to_datetime(data2_a_label_70['finish_time'], format='%M:%S')

In [512]:
# data2_a_label_80['start_time'] = pd.to_datetime(data2_a_label_80['start_time'], format='%M:%S')
# data2_a_label_80['finish_time'] = pd.to_datetime(data2_a_label_80['finish_time'], format='%M:%S')

In [513]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a)
display(data2_a_label_70)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,2020-11-11 14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,2020-11-11 14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,2020-11-11 14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,2020-11-11 14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,2020-11-11 15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,2020-11-11 15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,2020-11-11 15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,2020-11-11 15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


,start_time,finish_time,emotion_level
0,1900-01-01 00:03:27,1900-01-01 00:03:31,0
1,1900-01-01 00:03:41,1900-01-01 00:03:45,-2
2,1900-01-01 00:03:42,1900-01-01 00:03:46,-2
3,1900-01-01 00:03:43,1900-01-01 00:03:47,-2
4,1900-01-01 00:03:44,1900-01-01 00:03:48,-2
...,...,...,...
459,1900-01-01 00:18:06,1900-01-01 00:18:10,-2
460,1900-01-01 00:18:07,1900-01-01 00:18:11,-2
461,1900-01-01 00:18:08,1900-01-01 00:18:12,-2
462,1900-01-01 00:18:09,1900-01-01 00:18:13,-2


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [514]:
#data1_a_label_50['start_time'] = data1_a_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_50['finish_time'] = data1_a_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [515]:
#data1_a_label_60['start_time'] = data1_a_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_60['finish_time'] = data1_a_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [516]:
#data1_a_label_70['start_time'] = data1_a_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_70['finish_time'] = data1_a_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [517]:
#data1_a_label_80['start_time'] = data1_a_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_80['finish_time'] = data1_a_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [518]:
# data2_a_label_50['start_time'] = data2_a_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_a_label_50['finish_time'] = data2_a_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [519]:
# data2_a_label_60['start_time'] = data2_a_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_a_label_60['finish_time'] = data2_a_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [520]:
data2_a_label_70['start_time'] = data2_a_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_a_label_70['finish_time'] = data2_a_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [521]:
# data2_a_label_80['start_time'] = data2_a_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_a_label_80['finish_time'] = data2_a_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [522]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:11,2020-11-11 15:15:15,0
1,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
2,2020-11-11 15:15:26,2020-11-11 15:15:30,-2
3,2020-11-11 15:15:27,2020-11-11 15:15:31,-2
4,2020-11-11 15:15:28,2020-11-11 15:15:32,-2
...,...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
460,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
461,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
462,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [523]:
#datetime化したラベルデータを保存
# data1_a_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_a_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_a_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_a_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_a_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_a_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_a_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_a_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [524]:
#datetime化した加速度及び角速度データを保存
# data1_a.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_a.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [525]:
#時刻をインデックスに
#data1_a.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

data2_a.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [526]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 14:57:59.245,ags,9801,898,-258,340,1,913,9845.433916,974.253560
2020-11-11 14:57:59.250,ags,9911,592,-290,291,44,559,9932.899124,631.742036
2020-11-11 14:57:59.255,ags,9996,431,-268,218,-188,279,10008.876111,400.885270
2020-11-11 14:57:59.260,ags,10150,453,-280,114,-84,-75,10163.961285,160.240444
2020-11-11 14:57:59.265,ags,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...
2020-11-11 15:37:07.780,ags,10184,356,313,-318,860,-20,10195.026287,917.128126
2020-11-11 15:37:07.785,ags,10216,282,230,-190,1006,-117,10222.479151,1030.448931
2020-11-11 15:37:07.790,ags,10087,336,339,-68,1116,-208,10098.286290,1137.252830


In [527]:
#data1_a_label_50["start_time"] = data1_a_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_50["finish_time"] = data1_a_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [528]:
#data1_a_label_60["start_time"] = data1_a_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_60["finish_time"] = data1_a_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [529]:
#data1_a_label_70["start_time"] = data1_a_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_70["finish_time"] = data1_a_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [530]:
#data1_a_label_80["start_time"] = data1_a_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_80["finish_time"] = data1_a_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [531]:
# data2_a_label_50["start_time"] = data2_a_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_a_label_50["finish_time"] = data2_a_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [532]:
# data2_a_label_60["start_time"] = data2_a_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_a_label_60["finish_time"] = data2_a_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [533]:
data2_a_label_70["start_time"] = data2_a_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_a_label_70["finish_time"] = data2_a_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [534]:
# data2_a_label_80["start_time"] = data2_a_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_a_label_80["finish_time"] = data2_a_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [535]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:11,2020-11-11 15:15:15,0
1,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
2,2020-11-11 15:15:26,2020-11-11 15:15:30,-2
3,2020-11-11 15:15:27,2020-11-11 15:15:31,-2
4,2020-11-11 15:15:28,2020-11-11 15:15:32,-2
...,...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
460,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
461,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
462,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [536]:
#startとfinishをリスト化
#data1_a_50_start_time = list(data1_a_label_50["start_time"])
#data1_a_50_finish_time = list(data1_a_label_50["finish_time"])

#data1_a_60_start_time = list(data1_a_label_60["start_time"])
#data1_a_60_finish_time = list(data1_a_label_60["finish_time"])

#data1_a_70_start_time = list(data1_a_label_70["start_time"])
#data1_a_70_finish_time = list(data1_a_label_70["finish_time"])

#data1_a_80_start_time = list(data1_a_label_80["start_time"])
#data1_a_80_finish_time = list(data1_a_label_80["finish_time"])

# data2_a_50_start_time = list(data2_a_label_50["start_time"])
# data2_a_50_finish_time = list(data2_a_label_50["finish_time"])

# data2_a_60_start_time = list(data2_a_label_60["start_time"])
# data2_a_60_finish_time = list(data2_a_label_60["finish_time"])

data2_a_70_start_time = list(data2_a_label_70["start_time"])
data2_a_70_finish_time = list(data2_a_label_70["finish_time"])

# data2_a_80_start_time = list(data2_a_label_80["start_time"])
# data2_a_80_finish_time = list(data2_a_label_80["finish_time"])

In [537]:
data2_a_70_start_time

['2020-11-11 15:15:11',
 '2020-11-11 15:15:25',
 '2020-11-11 15:15:26',
 '2020-11-11 15:15:27',
 '2020-11-11 15:15:28',
 '2020-11-11 15:15:29',
 '2020-11-11 15:15:30',
 '2020-11-11 15:15:31',
 '2020-11-11 15:15:32',
 '2020-11-11 15:15:33',
 '2020-11-11 15:15:34',
 '2020-11-11 15:15:35',
 '2020-11-11 15:15:36',
 '2020-11-11 15:15:37',
 '2020-11-11 15:15:38',
 '2020-11-11 15:15:39',
 '2020-11-11 15:15:40',
 '2020-11-11 15:15:41',
 '2020-11-11 15:15:42',
 '2020-11-11 15:15:43',
 '2020-11-11 15:15:44',
 '2020-11-11 15:15:53',
 '2020-11-11 15:15:54',
 '2020-11-11 15:16:08',
 '2020-11-11 15:16:09',
 '2020-11-11 15:16:10',
 '2020-11-11 15:16:11',
 '2020-11-11 15:16:12',
 '2020-11-11 15:16:13',
 '2020-11-11 15:16:14',
 '2020-11-11 15:16:15',
 '2020-11-11 15:16:16',
 '2020-11-11 15:16:17',
 '2020-11-11 15:16:18',
 '2020-11-11 15:16:19',
 '2020-11-11 15:16:20',
 '2020-11-11 15:16:21',
 '2020-11-11 15:16:22',
 '2020-11-11 15:16:23',
 '2020-11-11 15:16:24',
 '2020-11-11 15:16:25',
 '2020-11-11 15:

In [538]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

#data1_a_record_1 = sensor_segmentation_order(data1_a, data1_a_50_start_time, data1_a_50_finish_time, data1_a_label_50)
#data1_a_record_2 = sensor_segmentation_order(data1_a, data1_a_60_start_time, data1_a_60_finish_time, data1_a_label_60)
#data1_a_record_3 = sensor_segmentation_order(data1_a, data1_a_70_start_time, data1_a_70_finish_time, data1_a_label_70)
#data1_a_record_4 = sensor_segmentation_order(data1_a, data1_a_80_start_time, data1_a_80_finish_time, data1_a_label_80)

# data2_a_record_1 = sensor_segmentation_order(data2_a, data2_a_50_start_time, data2_a_50_finish_time, data2_a_label_50)
# data2_a_record_2 = sensor_segmentation_order(data2_a, data2_a_60_start_time, data2_a_60_finish_time, data2_a_label_60)
data2_a_record_3 = sensor_segmentation_order(data2_a, data2_a_70_start_time, data2_a_70_finish_time, data2_a_label_70)
# data2_a_record_4 = sensor_segmentation_order(data2_a, data2_a_80_start_time, data2_a_80_finish_time, data2_a_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [539]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

#data1_a_50_record_mean = acc_gyr_mean_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_mean = acc_gyr_mean_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_mean = acc_gyr_mean_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_mean = acc_gyr_mean_order(data1_a_record_4, data1_a_label_80)

# data2_a_50_record_mean = acc_gyr_mean_order(data2_a_record_1, data2_a_label_50)
# data2_a_60_record_mean = acc_gyr_mean_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_mean = acc_gyr_mean_order(data2_a_record_3, data2_a_label_70)
# data2_a_80_record_mean = acc_gyr_mean_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_mean
# b = data1_a_60_record_mean
# c = data1_a_70_record_mean
# d = data1_a_80_record_mean

# e = data2_a_50_record_mean
# f = data2_a_60_record_mean
# g = data2_a_70_record_mean
# h = data2_a_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [540]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

#data1_a_50_record_var = acc_gyr_var_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_var = acc_gyr_var_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_var = acc_gyr_var_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_var = acc_gyr_var_order(data1_a_record_4, data1_a_label_80)

# data2_a_50_record_var = acc_gyr_var_order(data2_a_record_1, data2_a_label_50)
# data2_a_60_record_var = acc_gyr_var_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_var = acc_gyr_var_order(data2_a_record_3, data2_a_label_70)
# data2_a_80_record_var = acc_gyr_var_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_var
# b = data1_a_60_record_var
# c = data1_a_70_record_var
# d = data1_a_80_record_var

# e = data2_a_50_record_var
# f = data2_a_60_record_var
# g = data2_a_70_record_var
# h = data2_a_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [541]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

# data1_a_50_record_max = acc_gyr_max_order(data1_a_record_1, data1_a_label_50)
# data1_a_60_record_max = acc_gyr_max_order(data1_a_record_2, data1_a_label_60)
# data1_a_70_record_max = acc_gyr_max_order(data1_a_record_3, data1_a_label_70)
# data1_a_80_record_max = acc_gyr_max_order(data1_a_record_4, data1_a_label_80)

# data2_a_50_record_max = acc_gyr_max_order(data2_a_record_1, data2_a_label_50)
# data2_a_60_record_max = acc_gyr_max_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_max = acc_gyr_max_order(data2_a_record_3, data2_a_label_70)
# data2_a_80_record_max = acc_gyr_max_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_max
# b = data1_a_60_record_max
# c = data1_a_70_record_max
# d = data1_a_80_record_max

# e = data2_a_50_record_max
# f = data2_a_60_record_max
# g = data2_a_70_record_max
# h = data2_a_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [542]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

#data1_a_50_record_min = acc_gyr_min_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_min = acc_gyr_min_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_min = acc_gyr_min_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_min = acc_gyr_min_order(data1_a_record_4, data1_a_label_80)

# data2_a_50_record_min = acc_gyr_min_order(data2_a_record_1, data2_a_label_50)
# data2_a_60_record_min = acc_gyr_min_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_min = acc_gyr_min_order(data2_a_record_3, data2_a_label_70)
# data2_a_80_record_min = acc_gyr_min_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_min
# b = data1_a_60_record_min
# c = data1_a_70_record_min
# d = data1_a_80_record_min

# e = data2_a_50_record_min
# f = data2_a_60_record_min
# g = data2_a_70_record_min
# h = data2_a_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [543]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

# data1_a_50_record_fq_amp_max = fq_amp_max_order(data1_a_record_1, data1_a_label_50)
# data1_a_60_record_fq_amp_max = fq_amp_max_order(data1_a_record_2, data1_a_label_60)
# data1_a_70_record_fq_amp_max = fq_amp_max_order(data1_a_record_3, data1_a_label_70)
# data1_a_80_record_fq_amp_max = fq_amp_max_order(data1_a_record_4, data1_a_label_80)

# data2_a_50_record_fq_amp_max = fq_amp_max_order(data2_a_record_1, data2_a_label_50)
# data2_a_60_record_fq_amp_max = fq_amp_max_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_fq_amp_max = fq_amp_max_order(data2_a_record_3, data2_a_label_70)
# data2_a_80_record_fq_amp_max = fq_amp_max_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_fq_amp_max
# b = data1_a_60_record_fq_amp_max
# c = data1_a_70_record_fq_amp_max
# d = data1_a_80_record_fq_amp_max

# e = data2_a_50_record_fq_amp_max
# f = data2_a_60_record_fq_amp_max
# g = data2_a_70_record_fq_amp_max
# h = data2_a_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [544]:
#全特徴量の行列の導出
# data1_a_50_record_all_feature = pd.concat([data1_a_50_record_mean, data1_a_50_record_var, data1_a_50_record_max, data1_a_50_record_min, data1_a_50_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_60_record_all_feature = pd.concat([data1_a_60_record_mean, data1_a_60_record_var, data1_a_60_record_max, data1_a_60_record_min, data1_a_60_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_70_record_all_feature = pd.concat([data1_a_70_record_mean, data1_a_70_record_var, data1_a_70_record_max, data1_a_70_record_min, data1_a_70_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_80_record_all_feature = pd.concat([data1_a_80_record_mean, data1_a_80_record_var, data1_a_80_record_max, data1_a_80_record_min, data1_a_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_a_50_record_all_feature = pd.concat([data2_a_50_record_mean, data2_a_50_record_var, data2_a_50_record_max, data2_a_50_record_min, data2_a_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_a_60_record_all_feature = pd.concat([data2_a_60_record_mean, data2_a_60_record_var, data2_a_60_record_max, data2_a_60_record_min, data2_a_60_record_fq_amp_max], axis=1, ignore_index=False)
data2_a_70_record_all_feature = pd.concat([data2_a_70_record_mean, data2_a_70_record_var, data2_a_70_record_max, data2_a_70_record_min, data2_a_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_a_80_record_all_feature = pd.concat([data2_a_80_record_mean, data2_a_80_record_var, data2_a_80_record_max, data2_a_80_record_min, data2_a_80_record_fq_amp_max], axis=1, ignore_index=False)

In [545]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data2_a_70_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,6413.464,1748.613,-7641.666,41.576,104.271,-31.387,36274.960704,4895.439231,46043.638444,11048.174224,...,4970,1557,-8840,-433,-1926,-659,8.610568,61.267628,0.391389,369.979335
1,6585.332,1688.401,-7515.517,21.096,17.128,-34.625,6752.361776,1485.486199,16085.719711,4920.556784,...,6378,1574,-8015,-177,-962,-379,7.827789,46.782133,0.782779,149.596783
2,6558.258,1688.831,-7538.994,15.447,31.227,-22.915,4754.205436,1328.646439,10507.859964,3979.821191,...,6378,1547,-7910,-177,-767,-287,8.219178,31.594310,1.174168,114.267170
3,6579.836,1686.730,-7520.997,12.174,97.806,-16.283,11794.593104,1324.087100,16338.822991,3926.463724,...,6378,1547,-7910,-190,-767,-251,9.393346,42.356923,0.782779,86.940697
4,6646.337,1682.214,-7460.126,21.330,77.968,-26.278,22926.355431,1446.054204,30619.722124,4981.431100,...,6378,1547,-7966,-190,-1505,-360,9.784736,38.452860,1.174168,84.342775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,4680.155,1092.147,-8909.073,22.956,42.522,-27.508,13713.880975,1162.153391,9853.445671,1728.512064,...,4401,978,-9314,-104,-797,-226,9.393346,50.237792,1.174168,88.251329
460,4677.243,1091.027,-8910.840,21.695,54.193,-27.739,15401.071951,1179.686271,11202.804400,1915.049975,...,4323,966,-9372,-135,-797,-226,9.001957,38.951199,1.174168,65.250229
461,4637.278,1092.880,-8928.580,20.961,27.404,-31.993,24031.314716,1472.921600,19551.475600,2308.777479,...,4147,954,-9673,-196,-3547,-336,12.133072,32.959010,0.782779,75.221671
462,4591.915,1097.311,-8954.301,19.446,36.201,-28.432,17363.933775,1470.200279,17303.246399,2150.831084,...,4147,954,-9673,-196,-3547,-336,11.350294,35.223713,2.739726,38.596097


In [546]:
#各特徴量データの保存
# data1_a_50_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
# data1_a_60_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
# data1_a_70_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
# data1_a_80_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

# data2_a_50_record_all_feature.to_csv('feature/A/data2_a_50_record_all_feature.csv', index=False)
# data2_a_60_record_all_feature.to_csv('feature/A/data2_a_60_record_all_feature.csv', index=False)
data2_a_70_record_all_feature.to_csv('feature/A/data2_a_70_record_all_feature.csv', index=False)
# data2_a_80_record_all_feature.to_csv('feature/A/data2_a_80_record_all_feature.csv', index=False)

In [547]:
data2_a_label_70

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:11,2020-11-11 15:15:15,0
1,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
2,2020-11-11 15:15:26,2020-11-11 15:15:30,-2
3,2020-11-11 15:15:27,2020-11-11 15:15:31,-2
4,2020-11-11 15:15:28,2020-11-11 15:15:32,-2
...,...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
460,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
461,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
462,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [548]:
# data1_a_label_50_time = data1_a_label_50.drop("emotion_level", axis=1)
# data1_a_label_60_time = data1_a_label_60.drop("emotion_level", axis=1)
# data1_a_label_70_time = data1_a_label_70.drop("emotion_level", axis=1)
# data1_a_label_80_time = data1_a_label_80.drop("emotion_level", axis=1)

# data2_a_label_50_time = data2_a_label_50.drop("emotion_level", axis=1)
# data2_a_label_60_time = data2_a_label_60.drop("emotion_level", axis=1)
data2_a_label_70_time = data2_a_label_70.drop("emotion_level", axis=1)
# data2_a_label_80_time = data2_a_label_80.drop("emotion_level", axis=1)

In [549]:
data2_a_label_70_time

,start_time,finish_time
0,2020-11-11 15:15:11,2020-11-11 15:15:15
1,2020-11-11 15:15:25,2020-11-11 15:15:29
2,2020-11-11 15:15:26,2020-11-11 15:15:30
3,2020-11-11 15:15:27,2020-11-11 15:15:31
4,2020-11-11 15:15:28,2020-11-11 15:15:32
...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54
460,2020-11-11 15:29:51,2020-11-11 15:29:55
461,2020-11-11 15:29:52,2020-11-11 15:29:56
462,2020-11-11 15:29:53,2020-11-11 15:29:57


In [550]:
# data1_a_50_record_all_feature = pd.concat([data1_a_label_50_time, data1_a_50_record_all_feature], axis=1)
# data1_a_60_record_all_feature = pd.concat([data1_a_label_60_time, data1_a_60_record_all_feature], axis=1)
# data1_a_70_record_all_feature = pd.concat([data1_a_label_70_time, data1_a_70_record_all_feature], axis=1)
# data1_a_80_record_all_feature = pd.concat([data1_a_label_80_time, data1_a_80_record_all_feature], axis=1)

# data2_a_50_record_all_feature = pd.concat([data2_a_label_50_time, data2_a_50_record_all_feature], axis=1)
# data2_a_60_record_all_feature = pd.concat([data2_a_label_60_time, data2_a_60_record_all_feature], axis=1)
data2_a_70_record_all_feature = pd.concat([data2_a_label_70_time, data2_a_70_record_all_feature], axis=1)
# data2_a_80_record_all_feature = pd.concat([data2_a_label_80_time, data2_a_80_record_all_feature], axis=1)

In [551]:
data2_a_70_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:11,2020-11-11 15:15:15,6413.464,1748.613,-7641.666,41.576,104.271,-31.387,36274.960704,4895.439231,...,4970,1557,-8840,-433,-1926,-659,8.610568,61.267628,0.391389,369.979335
1,2020-11-11 15:15:25,2020-11-11 15:15:29,6585.332,1688.401,-7515.517,21.096,17.128,-34.625,6752.361776,1485.486199,...,6378,1574,-8015,-177,-962,-379,7.827789,46.782133,0.782779,149.596783
2,2020-11-11 15:15:26,2020-11-11 15:15:30,6558.258,1688.831,-7538.994,15.447,31.227,-22.915,4754.205436,1328.646439,...,6378,1547,-7910,-177,-767,-287,8.219178,31.594310,1.174168,114.267170
3,2020-11-11 15:15:27,2020-11-11 15:15:31,6579.836,1686.730,-7520.997,12.174,97.806,-16.283,11794.593104,1324.087100,...,6378,1547,-7910,-190,-767,-251,9.393346,42.356923,0.782779,86.940697
4,2020-11-11 15:15:28,2020-11-11 15:15:32,6646.337,1682.214,-7460.126,21.330,77.968,-26.278,22926.355431,1446.054204,...,6378,1547,-7966,-190,-1505,-360,9.784736,38.452860,1.174168,84.342775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54,4680.155,1092.147,-8909.073,22.956,42.522,-27.508,13713.880975,1162.153391,...,4401,978,-9314,-104,-797,-226,9.393346,50.237792,1.174168,88.251329
460,2020-11-11 15:29:51,2020-11-11 15:29:55,4677.243,1091.027,-8910.840,21.695,54.193,-27.739,15401.071951,1179.686271,...,4323,966,-9372,-135,-797,-226,9.001957,38.951199,1.174168,65.250229
461,2020-11-11 15:29:52,2020-11-11 15:29:56,4637.278,1092.880,-8928.580,20.961,27.404,-31.993,24031.314716,1472.921600,...,4147,954,-9673,-196,-3547,-336,12.133072,32.959010,0.782779,75.221671
462,2020-11-11 15:29:53,2020-11-11 15:29:57,4591.915,1097.311,-8954.301,19.446,36.201,-28.432,17363.933775,1470.200279,...,4147,954,-9673,-196,-3547,-336,11.350294,35.223713,2.739726,38.596097


In [552]:
# data1_a_50_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
# data1_a_60_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
# data1_a_70_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
# data1_a_80_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

# data2_a_50_record_all_feature.to_csv('feature/A/data2_a_50_record_all_feature.csv', index=False)
# data2_a_60_record_all_feature.to_csv('feature/A/data2_a_60_record_all_feature.csv', index=False)
data2_a_70_record_all_feature.to_csv('feature/A/data2_a_70_record_all_feature.csv', index=False)
# data2_a_80_record_all_feature.to_csv('feature/A/data2_a_80_record_all_feature.csv', index=False)

In [553]:
data2_a_70_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:11,2020-11-11 15:15:15,6413.464,1748.613,-7641.666,41.576,104.271,-31.387,36274.960704,4895.439231,...,4970,1557,-8840,-433,-1926,-659,8.610568,61.267628,0.391389,369.979335
1,2020-11-11 15:15:25,2020-11-11 15:15:29,6585.332,1688.401,-7515.517,21.096,17.128,-34.625,6752.361776,1485.486199,...,6378,1574,-8015,-177,-962,-379,7.827789,46.782133,0.782779,149.596783
2,2020-11-11 15:15:26,2020-11-11 15:15:30,6558.258,1688.831,-7538.994,15.447,31.227,-22.915,4754.205436,1328.646439,...,6378,1547,-7910,-177,-767,-287,8.219178,31.594310,1.174168,114.267170
3,2020-11-11 15:15:27,2020-11-11 15:15:31,6579.836,1686.730,-7520.997,12.174,97.806,-16.283,11794.593104,1324.087100,...,6378,1547,-7910,-190,-767,-251,9.393346,42.356923,0.782779,86.940697
4,2020-11-11 15:15:28,2020-11-11 15:15:32,6646.337,1682.214,-7460.126,21.330,77.968,-26.278,22926.355431,1446.054204,...,6378,1547,-7966,-190,-1505,-360,9.784736,38.452860,1.174168,84.342775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,2020-11-11 15:29:50,2020-11-11 15:29:54,4680.155,1092.147,-8909.073,22.956,42.522,-27.508,13713.880975,1162.153391,...,4401,978,-9314,-104,-797,-226,9.393346,50.237792,1.174168,88.251329
460,2020-11-11 15:29:51,2020-11-11 15:29:55,4677.243,1091.027,-8910.840,21.695,54.193,-27.739,15401.071951,1179.686271,...,4323,966,-9372,-135,-797,-226,9.001957,38.951199,1.174168,65.250229
461,2020-11-11 15:29:52,2020-11-11 15:29:56,4637.278,1092.880,-8928.580,20.961,27.404,-31.993,24031.314716,1472.921600,...,4147,954,-9673,-196,-3547,-336,12.133072,32.959010,0.782779,75.221671
462,2020-11-11 15:29:53,2020-11-11 15:29:57,4591.915,1097.311,-8954.301,19.446,36.201,-28.432,17363.933775,1470.200279,...,4147,954,-9673,-196,-3547,-336,11.350294,35.223713,2.739726,38.596097
